# Introduction to Computer Vision with OpenCV, YOLOv11, and Stable Diffusion


## Objectives

- Understand the basics of OpenCV and image handling in Python  
- Apply common image filters with OpenCV  
- Perform real‑time object detection with **YOLOv11**  
- Generate images from text prompts using **Stable Diffusion 3** via the **Diffusers** library  


In [ ]:

# 🛠️ Setup (uncomment to run in a fresh environment)
# Note: A CUDA‑enabled GPU is strongly recommended for YOLOv11 and Stable Diffusion.
# !pip install opencv-python ultralytics diffusers transformers accelerate torch torchvision matplotlib pillow


## 1. Basic Introduction to OpenCV

In [ ]:

import cv2
import matplotlib.pyplot as plt

# Load an example image
img = cv2.imread('sample.jpg')  # ⬅️ Replace with your own image path
if img is None:
    raise FileNotFoundError("Please place an image named 'sample.jpg' in the notebook directory or edit the path.")

# Convert BGR (OpenCV default) to RGB for Matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_rgb)
plt.title('Original Image')
plt.axis('off')
plt.show()


OpenCV loads images in **BGR** order. We converted to **RGB** here so the colors display correctly in Matplotlib.

## 2. Image Filtering with OpenCV

In [ ]:

# Apply common filters
gray   = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur   = cv2.GaussianBlur(img_rgb, (5, 5), 0)
median = cv2.medianBlur(img_rgb, 5)
edges  = cv2.Canny(img, 100, 200)

# Display results
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
ax[0, 0].imshow(gray, cmap='gray');  ax[0, 0].set_title('Grayscale');     ax[0, 0].axis('off')
ax[0, 1].imshow(blur);              ax[0, 1].set_title('Gaussian Blur'); ax[0, 1].axis('off')
ax[1, 0].imshow(median);            ax[1, 0].set_title('Median Blur');    ax[1, 0].axis('off')
ax[1, 1].imshow(edges, cmap='gray');ax[1, 1].set_title('Canny Edges');    ax[1, 1].axis('off')
plt.tight_layout()


The filters above demonstrate noise reduction (`GaussianBlur`, `medianBlur`) and edge detection (`Canny`). Experiment with kernel sizes and thresholds!

## 3. Object Detection with YOLOv11

In [ ]:

from ultralytics import YOLO

# Load a pretrained YOLOv11n model (nano variant for speed)
model = YOLO('yolov11n.pt')  # Automatically downloads weights the first time

results = model('sample.jpg')  # Path to image or list/ndarray
results[0].show()  # In notebooks this displays inline



* `results` contains predicted bounding boxes, confidence scores, and class labels.  
* Try different model variants like `yolov11s.pt`, `yolov11m.pt`, or the high‑accuracy `yolov11x.pt`.  
* To run on video streams, simply pass a video file path or a webcam index (e.g., `0`) to `model()`.  


## 4. Text‑to‑Image Generation with Stable Diffusion 3

In [ ]:

import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = StableDiffusionPipeline.from_pretrained(
    'stabilityai/stable-diffusion-3-base',
    torch_dtype=torch.float16 if device == 'cuda' else torch.float32
).to(device)

prompt = "A futuristic cityscape at dusk, cinematic, ultra realistic"
image = pipe(prompt, num_inference_steps=30, guidance_scale=8.0).images[0]

plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')
plt.title(prompt)
plt.show()



> **Tip:** For lower‑memory GPUs you can enable *memory‑efficient attention* with `pipe.enable_model_cpu_offload()` or use a quantized checkpoint such as **SD3‑Medium‑Turbo**.



## 5. Next Steps

- Fine‑tune **YOLOv11** on your own dataset with `model.train(data='data.yaml')`  
- Experiment with different Stable Diffusion models (`sd3.5`, `sdxl`) and creative prompts  
- Combine detection and generation: detect objects, then inpaint or replace them with Stable Diffusion!  

---

### References

- [Ultralytics YOLOv11 Documentation](https://docs.ultralytics.com/models/yolo11/)  
- [Hugging Face **Diffusers** Library](https://github.com/huggingface/diffusers)  
- [Stable Diffusion 3 Announcement](https://huggingface.co/blog/sd3)  
